In [1]:
import os
from process.resize import resize
import cv2
from process.crop_face import img_face

In [2]:
files = []
for r, d, f in os.walk('image/'):
    for file in f:
        if '.jpg' in file:
            exact_path = r + file
            files.append(exact_path)

In [3]:
files

['image/it_6.jpg',
 'image/wave_7.jpg',
 'image/it_5.jpg',
 'image/oak_2.jpg',
 'image/wave_8.jpg',
 'image/beam_4.jpg',
 'image/beam_3.jpg',
 'image/oak_1.jpg']

In [4]:
images = []
face_done = []
detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

for i in files:
    image = cv2.imread(i)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray, 1.1, 4)
    
    if type(faces) != tuple:
        for x, y, w, h in faces:
            img = image[x:x+w, y:y+h]
            print(i + " can find face.")
            face_done.append(i)

            images.append(img)

image/it_6.jpg can find face.
image/it_5.jpg can find face.
image/oak_2.jpg can find face.
image/beam_4.jpg can find face.
image/beam_3.jpg can find face.
image/oak_1.jpg can find face.


In [5]:
from Model.facenet_basemodel import FaceNet
model = FaceNet().loadModel('./Model/facenet_weights.h5')

Ready


In [6]:
representations = []
for img in images:
    for i in face_done:
        img_re = resize(img)
        embedding = model.predict(img_re)[0, :]
    
        representation = []
        representation.append(i)
        representation.append(embedding)
        representations.append(representation)

In [7]:
embedding.shape

(128,)

Synthetic Data

In [8]:
import random

In [9]:
for i in range(len(representations), 1000000):
    key = 'image/'

Spotify Annoy

In [10]:
from annoy import AnnoyIndex

In [39]:
embedding_size =128
t = AnnoyIndex(embedding_size, 'angular')

for i in range(0, len(representations)):
    representation = representations[i]
    img_path = representation[0]
    embedding = representation[1]
    
    t.add_item(i, embedding)
    
t.build(3)

True

In [40]:
t.save('result.ann')

True

In [41]:
n = AnnoyIndex(128, 'angular')
n.load('result.ann')

True

In [58]:
path = './image/oak_1.jpg'
detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

img = cv2.imread(path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = detector.detectMultiScale(gray, 1.1, 4)
print(faces)
for x, y, w, h in faces:
    img_crop = img[x:x+w, y:y+h]

img_re = cv2.resize(img_crop,(160,160))
img_re = img_re[None,:,:,:]

result = model.predict(img_re).reshape(-1, 1)
idx = n.get_nns_by_vector(result, 1)

print(idx)
for i in idx:
    print(representations[i])

[[198 191 249 249]]
[30]
['image/it_6.jpg', array([-0.7921088 ,  0.2920169 ,  0.32483315,  1.2468144 ,  0.8860202 ,
        1.0491496 ,  1.1581718 , -0.6650983 , -0.60243326, -1.581208  ,
        0.632371  , -1.408     , -0.93279684,  0.87667084, -1.414259  ,
       -1.1295882 ,  1.3431023 , -2.032857  , -0.69040644, -0.4741681 ,
       -0.19550328, -2.2388253 ,  1.5319884 ,  1.4266689 ,  0.6042202 ,
        1.9383725 ,  2.0091598 ,  0.16813639, -0.42985165, -2.310195  ,
        1.1581346 ,  0.9502694 ,  0.10712388, -0.1393038 , -0.6261446 ,
        0.7705466 ,  1.1596378 , -0.7261128 ,  1.4950674 ,  0.36686623,
       -0.45526722, -0.66222274, -0.43239918, -0.7374906 , -0.6629798 ,
        1.8899657 ,  0.7332998 , -0.03091846, -1.0237244 , -0.99224126,
        1.3079667 ,  1.1786598 ,  1.4535798 ,  0.36983395,  0.63690835,
        4.5929832 , -0.04466813, -2.0361488 ,  2.2122607 ,  1.6630789 ,
       -0.19632101,  0.17697594,  1.7501659 , -0.07061014, -0.510817  ,
       -0.18340419, 

In [59]:
representations[30]

['image/it_6.jpg',
 array([-0.7921088 ,  0.2920169 ,  0.32483315,  1.2468144 ,  0.8860202 ,
         1.0491496 ,  1.1581718 , -0.6650983 , -0.60243326, -1.581208  ,
         0.632371  , -1.408     , -0.93279684,  0.87667084, -1.414259  ,
        -1.1295882 ,  1.3431023 , -2.032857  , -0.69040644, -0.4741681 ,
        -0.19550328, -2.2388253 ,  1.5319884 ,  1.4266689 ,  0.6042202 ,
         1.9383725 ,  2.0091598 ,  0.16813639, -0.42985165, -2.310195  ,
         1.1581346 ,  0.9502694 ,  0.10712388, -0.1393038 , -0.6261446 ,
         0.7705466 ,  1.1596378 , -0.7261128 ,  1.4950674 ,  0.36686623,
        -0.45526722, -0.66222274, -0.43239918, -0.7374906 , -0.6629798 ,
         1.8899657 ,  0.7332998 , -0.03091846, -1.0237244 , -0.99224126,
         1.3079667 ,  1.1786598 ,  1.4535798 ,  0.36983395,  0.63690835,
         4.5929832 , -0.04466813, -2.0361488 ,  2.2122607 ,  1.6630789 ,
        -0.19632101,  0.17697594,  1.7501659 , -0.07061014, -0.510817  ,
        -0.18340419,  0.6102153 